In [2]:
from scipy.special import j1
from scipy.optimize import brentq
from sage.all import *
import struct

DR = RealField(52)

DD = RealField(157)

def double_to_hex(f):
    # Converts Python float (f64) to hex string
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)


In [29]:
# from scipy.special import j1, jvp
# from scipy.optimize import brentq
from mpmath import mp, mpf, findroot, j1, besselj
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")
    
# Use Sage's RDF for high-precision floats
R120 = RealField(120)

# List of zeros
zeros = []

# Step size to detect sign changes
# step = R120(0.001)
# x = R120(0.0)

mp.prec = 150

step = mpf("0.001")
epsilon = mpf("1e-35")
x = mpf("0.0")

def j1_prime(x):
    return diff(lambda t: besselj(1, t), x)

previous_zero = R120(0)
# epsilon = 1e-12
j1_zeros = []

# while x < 77:
#     if j1(float(x)) * j1(float(x + step)) < 0:  # Sign change => zero in interval
#         previous_zero = brentq(j1, x, x + step)
#         j1_zeros.append(R120(previous_zero))
#     if abs(x - round(x)) < epsilon:
#         zeros.append(R120(previous_zero))
#     x += step

while x < mpf("76.0"):
    f1 = besselj(1, x)
    f2 = besselj(1, x + step)
    if f1 * f2 < 0:
        zero = findroot(lambda t: j1(t), (x, x + step), solver='bisect', tol=mp.mpf("1e-41"))
        previous_zero = zero
        j1_zeros.append(zero)
    if previous_zero is not None and abs(x - mpf(f'{round(x)}')) < epsilon:
        zeros.append(previous_zero)
    x += step

j1_extrema = []

# x = R120(0.0)
# while x < 77:
#     dfx = jvp(1, float(x), n=1)  # Derivative of J1
#     dfx_next = jvp(1, float(x + step), n=1)
#     if dfx * dfx_next < 0:
#         extremum = brentq(lambda x: jvp(1, x, n=1), float(x), float(x + step))
#         j1_extrema.append(R120(extremum))
#     x += step

x = mpf("0.0")
while x < mpf("76.0"):
    d1 = mp.diff(lambda t: j1(t), x)
    d2 = mp.diff(lambda t: j1(t), x + step)
    if d1 * d2 < 0:
        extremum = findroot(lambda t: mp.diff(lambda u: j1(u), t), (x, x + step), solver='bisect', tol=mp.mpf("1e-41"))
        j1_extrema.append(extremum)
    x += step

# Print results
for i, z in enumerate(j1_zeros):
    print(f"Zero {i+1}: x ≈ {z}")

print("Extrema (peaks/valleys) of J1(x):")
for e in j1_extrema:
    print(f"nExtrema: {e}")

j1_zeros.extend(j1_extrema)

j1_zeros = sorted(j1_zeros)

# Print results
for i, z in enumerate(j1_zeros):
    print(f"Peak or zero {i+1}: x ≈ {z}")

print("")

print("pub(crate) static J1_ZEROS: [(u64, u64); 48] = [")
print(f"(0x0, 0x0),")
for z in j1_zeros:
    k = split_double_double(z)
    hi = double_to_hex(k[1])
    lo = double_to_hex(k[0])
    print(f"({lo}, {hi}),")
    
print("];")

from mpmath import *
from sage.all import *

from mpmath import mp, j1, taylor, expm1

prev_zero = 0

CDR = RealField(120)

var('x')

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffs(poly, x0):
    print("J1TaylorExtendedSeries {")
    print_double_double("a0: ", poly[0])
    print_double_double("a1: ", poly[1])
    print_double_double("a2: ", poly[2])
    print_double_double("a3: ", poly[3])
    print_double_double("a4: ", poly[4])
    print_double_double("a5: ", poly[5])
    print_double_double("a6: ", poly[6])
    print_double_double("a7: ", poly[7])
    print_double_double("a8: ", poly[8])
    print_double_double("a9: ", poly[9])
    print_double_double("a10: ", poly[10])
    print_double_double("a11: ", poly[11])
    print_double_double("a12: ", poly[12])
    print_double_double("a13: ", poly[13])
    print_double_double("a14: ", poly[14])
    print_double_double("a15: ", poly[15])
    print("c: [")
    for i in range(16, 24):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")
    print("},")

def print_expansion_at_0():
    print(f"pub(crate) static J1_MACLAURIN_SERIES: J1MaclaurinSeries = J1MaclaurinSeries {{")
    from mpmath import mp, j1, taylor, expm1
    poly = taylor(lambda val: j1(val), 0, 48)
    # print(poly)
    real_i = 0
    print_double_double("a0: ", DD(poly[1]))
    print_double_double("a1: ", DD(poly[3]))
    print_double_double("a2: ", DD(poly[5]))
    print_double_double("a3: ", DD(poly[7]))
    print_double_double("a4: ", DD(poly[9]))
    print("series: [")
    for i in range(11, 48, 2):
        print(f"{double_to_hex(poly[i])},")
        real_i = real_i + 1
    print("],")

    print("};")

mp.prec = 180

print_expansion_at_0()

print(f"pub(crate) static J1_COEFFS: [J1TaylorExtendedSeries; {len(j1_zeros)}] = [")

for i in range(0, len(j1_zeros)):
    k_range = j1_zeros[i]
    range_diff = k_range - prev_zero
    g_c = 1

    x0 = mp.mpf(k_range)
    from mpmath import mp, j1, taylor
    poly = taylor(lambda val: j1(val), x0, 25)
    print_taylor_coeffs(poly, CDR(k_range))
    prev_zero = j1_zeros[i]

print("];")


Zero 1: x ≈ 3.8317059702075123156144358863081605638606567
Zero 2: x ≈ 7.0155866698156187535370499814765249743343443
Zero 3: x ≈ 10.173468135062722077185711776775844057409686
Zero 4: x ≈ 13.323691936314223032393684126947876441816528
Zero 5: x ≈ 16.470630050877632812552460470989551841565766
Zero 6: x ≈ 19.615858510468242021125065884137509740170716
Zero 7: x ≈ 22.760084380592771898053005152182257271181704
Zero 8: x ≈ 25.903672087618382625495855445979874037559629
Zero 9: x ≈ 29.046828534916855066647819883531961126175072
Zero 10: x ≈ 32.189679910974403626622984104460369114464945
Zero 11: x ≈ 35.332307550083865102634479022519014605953498
Zero 12: x ≈ 38.474766234771615112052197557716612761601455
Zero 13: x ≈ 41.617094212814450885863516805060289043752123
Zero 14: x ≈ 44.759318997652821732779352713212144411096444
Zero 15: x ≈ 47.901460887185447121274008722507507056216055
Zero 16: x ≈ 51.043535183571509468733034633224060674066057
Zero 17: x ≈ 54.185553641061320532099966214533889117293725
Zero 1

In [4]:
def print_expansion_at_0_f():
    print(f"pub(crate) const J1_MACLAURIN_SERIES: [u64; 13] = [")
    from mpmath import mp, j1, taylor, expm1
    mp.prec = 53
    poly = taylor(lambda val: j1(val), 0, 46)
    # print(poly)
    real_i = 0
    z = 0
    for i in range(1, 46, 2):
        print(f"{double_to_hex(poly[i])},")
        real_i = real_i + 1
    print("];")

    print(f"poly {poly}")

print_expansion_at_0_f()

pub(crate) const J1_MACLAURIN_SERIES: [u64; 13] = [
0x3fe0000000000000,
0xbfb0000000000000,
0x3f65555555555555,
0xbf0c71c71c71c71c,
0x3ea6c16c16c16c17,
0xbe3845c8a0ce5129,
0x3dc27e4fb7789f5c,
0xbd4522a43f65486a,
0x3cc2c9758daf5cd0,
0xbc3ab81ea75fcdf4,
0x3baf17697cf1cf13,
0xbb1e2637bef9ff1a,
0x3a88bce58901a35e,
0xb9f165e7c2d153f3,
0x39553585cdcbfb10,
0xb8b69f7da8510bcd,
0x38154ad09e6a6576,
0xb771d028acb00491,
0x36caaae78f4066a6,
0xb621f72d8389b3a3,
0x3575e69de22df5ce,
0xb4c84564b82a1184,
0x34188f11edf3ed4c,
];
poly [mpf('0.0'), mpf('0.5'), mpf('0.0'), mpf('-0.0625'), mpf('0.0'), mpf('0.0026041666666666665'), mpf('0.0'), mpf('-5.4253472222222219e-5'), mpf('0.0'), mpf('6.781684027777778e-7'), mpf('0.0'), mpf('-5.6514033564814812e-9'), mpf('0.0'), mpf('3.3639305693342149e-11'), mpf('0.0'), mpf('-1.5017547184527747e-13'), mpf('0.0'), mpf('5.2144261057388011e-16'), mpf('0.0'), mpf('-1.4484516960385557e-18'), mpf('0.0'), mpf('3.2919356728148996e-21'), mpf('0.0'), mpf('-6.2347266530585217e-24'

In [58]:
# Taylor series for f32
mp.prec = 60
print(f"pub(crate) static J1F_COEFFS: [[u64; 15]; {len(j1_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffsf(poly, x0):
    print("[")
    for i in range(0, 15):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")

prev_zero = 0

for i in range(0, len(j1_zeros)):
    k_range = j1_zeros[i]
    range_diff = k_range - prev_zero
    g_c = 1

    x0 = mp.mpf(k_range)
    from mpmath import mp, j1, taylor, expm1
    # The j1 (Bessel J_1) function from mpmath will compute with mp.dps precision
    # The Taylor series computation will also respect mp.dps
    poly = taylor(lambda val: j1(val), x0, 23)
    # print(poly)
    print_taylor_coeffsf(poly, CDR(k_range))
    prev_zero = j1_zeros[i]

print("];")

pub(crate) static J1F_COEFFS: [[u64; 15]; 48] = [
[
0x3fe29ea3d19f035f,
0x0000000000000000,
0xbfca41115c5df243,
0x3f78d1448e6fed48,
0x3f8c441a2f9de22b,
0xbf386671c18b088a,
0xbf39e2504ddc7608,
0x3ee34ccbca0c75d1,
0x3eda4973784d1087,
0xbe81045322aaab45,
0xbe70fae0da6cdcef,
0x3e13546cef5ed00a,
0x3dfe5ee82e667708,
0xbd9ec80cc8b63c39,
0xbd83eb2e99629b44,
],
[
0x0000000000000000,
0xbfd9c6cf582cbf7f,
0x3faae8a39f51ad04,
0x3fab589d1da13905,
0xbf7537544c331da7,
0xbf624b3409959064,
0x3f26e4c2d5354224,
0x3f083a06e30c4109,
0xbec9799d4c9f2549,
0xbea33825cd2e2c16,
0x3e617069233e916c,
0x3e34569b22afc3c8,
0xbdf03b9e9651056a,
0xbdbec62310af5f52,
0x3d75ec84e47b6f4f,
],
[
0xbfd626ee83500bf2,
0x0000000000000000,
0x3fc55f6bec9ef962,
0xbf83d23336fd10e4,
0xbf88c77a983a0814,
0x3f45cdc98db1cbe2,
0x3f373576ff46ee3b,
0xbef2461447d7b423,
0xbed7b853456b6eaa,
0x3e90abfc68274a98,
0x3e6ea7a1ee26124d,
0xbe235c0413e01418,
0xbdfb5c5d512fbb00,
0x3daf4c5e26fd6e90,
0x3d81e4c43397be51,
],
[
0x0000000000000000,
0x3fd33518b38

In [22]:
from mpmath import mp, j1, taylor, linspace, chebyfit

# Set precision (decimal places)
mp.prec = 107

# Generate degree-46 Taylor series of J1(x) at x = 0
poly = taylor(lambda val: j1(val), mp.mpf('1.8411837813407134767373918293742463'), 25)

def polyeval(coeffs, x, center):
    """Evaluate Taylor polynomial centered at `center` using Horner's method."""
    u = x - center
    # result = mpf(0)
    # for c in reversed(coeffs):
    #     result = result * u + c
    # return result
    result = RealField(53)(0)
    for (index, c) in enumerate(reversed(coeffs)):
        if index < 8:
            result = RealField(53)(result) * RealField(53)(u) + RealField(53)(c)
        else:
            result = RealField(107)(result) * RealField(107)(u) + RealField(107)(c)
    return result

xs = linspace(0.991, 2, 1000)  # 1000 evenly spaced points
max_rel_error = 0
max_abs_error = 0
worst_x = None

for x in xs:
    true_val = j1(x)
    approx_val = polyeval(poly, x, DD('1.8411837813407134767373918293742463'))
    if true_val != 0:
        rel_error = abs((approx_val - true_val) / true_val)
        if rel_error > max_rel_error:
            max_rel_error = rel_error
            worst_x = x

print(f"Max relative error over [0, 1]: {max_rel_error}")
print(f"Occurs at x = {worst_x}")
worst_x = None

for x in xs:
    true_val = j1(x)
    approx_val = polyeval(poly, x, DD('1.8411837813407134767373918293742463'))
    abs_error = abs(approx_val - true_val)
    if abs_error > max_abs_error:
        max_abs_error = abs_error
        worst_x = x

print(f"Max abs error over [0, 1]: {max_rel_error}")
print(f"Occurs at x = {worst_x}")

print(double_to_hex(DD(max_rel_error)))
print(polyeval(poly, mp.mpf('0.9750000000213042'), mp.mpf('1.8411837813407134767373918293742463')))
def make_e(eps, p):
    return float((DD(1) + DD(2)**DD(-p)) / (DD(1) - eps - DD(2) ** DD(p + 1)*eps)) * 2**(-p)
print(double_to_hex(make_e(DD('1.192857060732688978535982892962e-26'), 55)))
print(polyeval(poly, DD('0.9218750000227204'), DD('1.8411837813407134767373918293742463')))

Max relative error over [0, 1]: 1.224074717441528028996975552824e-29
Occurs at x = 0.9909999999999999920063942226989
Max abs error over [0, 1]: 1.224074717441528028996975552824e-29
Occurs at x = 0.9909999999999999920063942226989
0x39ef08b2faac819b
0.4318209025803349383680360368051
0x3c800000003b1143
0.4136748745759469052209620580648
